# Modelagem de Tópicos com LDA, NMF e SVD
Neste notebook, vamos explorar a modelagem de tópicos utilizando três algoritmos diferentes: Latent Dirichlet Allocation (LDA), Non-negative Matrix Factorization (NMF), e Singular Value Decomposition (SVD). Nosso objetivo é encontrar o melhor valor de \( k \) (número de tópicos) para cada um desses algoritmos, utilizando métricas específicas a cada um. Por fim, vamos comparar os resultados de cada modelo.


In [1]:
# Libraries
import warnings

warnings.filterwarnings('ignore')
import re

import gensim
import matplotlib.pyplot as plt
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import seaborn as sns
from gensim import corpora
from gensim.models import CoherenceModel, LdaModel, LsiModel, Nmf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

## Modelagem de Tópicos com LDA
O **Latent Dirichlet Allocation (LDA)** é um modelo probabilístico que gera uma distribuição de tópicos para um conjunto de documentos. Vamos utilizar as métricas de **coerência de tópicos** e **perplexidade** para encontrar o melhor número de tópicos \( k \).



## Leitura dos dados

### Análise dos Resultados do LDA
Com base no gráfico de coerência, podemos selecionar o valor de \( k \) que maximiza a coerência dos tópicos. Também consideraremos a **perplexidade** para refinar a escolha de \( k \).


## Modelagem de Tópicos com NMF
O **Non-negative Matrix Factorization (NMF)** é um algoritmo de decomposição matricial que fatoriza a matriz de documentos em componentes não-negativos. Vamos otimizar \( k \) utilizando o **erro de reconstrução**.


### Análise dos Resultados do NMF
Aqui analisamos o **erro de reconstrução** para encontrar o melhor número de tópicos. Menores valores de erro indicam uma melhor qualidade na decomposição dos tópicos.


## Modelagem de Tópicos com SVD
O **Singular Value Decomposition (SVD)** é uma técnica de fatoração matricial. Vamos utilizar a **variância explicada** e **validação cruzada** para definir o número ideal de componentes \( k \).


### Análise dos Resultados do SVD
